In [1]:
# 필요한 라이브러리 임포트
import os
import json
import tiktoken
from openai import OpenAI
from typing import List, Dict, Any, Optional
import re

In [2]:
# OpenAI 클라이언트 설정
client = OpenAI(api_key=)

In [3]:
# [Cell 3] - 토크나이저 초기화 및 토큰 카운트 함수
def count_tokens(text: str) -> int:
    """텍스트의 토큰 수를 계산합니다."""
    encoding = tiktoken.encoding_for_model("gpt-4")
    return len(encoding.encode(text))

def split_into_chunks(text: str, max_tokens: int = 10000) -> List[str]:
    """텍스트를 지정된 토큰 수로 청크를 나눕니다."""
    encoding = tiktoken.encoding_for_model("gpt-4")
    tokens = encoding.encode(text)
    chunks = []
    current_chunk = []
    current_length = 0
    
    # 문장 단위로 텍스트 분할
    sentences = re.split(r'([.!?])\s+', text)
    
    for sentence in sentences:
        sentence_tokens = encoding.encode(sentence)
        sentence_length = len(sentence_tokens)
        
        if current_length + sentence_length > max_tokens:
            # 현재 청크가 최대 토큰 수를 초과하면 새로운 청크 시작
            chunks.append(encoding.decode(current_chunk))
            current_chunk = sentence_tokens
            current_length = sentence_length
        else:
            current_chunk.extend(sentence_tokens)
            current_length += sentence_length
    
    if current_chunk:
        chunks.append(encoding.decode(current_chunk))
    
    return chunks

In [4]:
# [Cell 4] - JSON 스키마 정의
SCRIPT_SCHEMA = {
    "title": "Play Title",
    "genre": "Drama",
    "characters": [
        {
            "name": str,
            "description": str,
            "extra": dict
        }
    ],
    "acts": [
        {
            "act_number": int,
            "scenes": [
                {
                    "scene_number": int,
                    "setting": {
                        "location": str,
                        "scene_time": str
                    },
                    "directions": str,
                    "extra": dict,
                    "dialogue": [
                        {
                            "character": str,
                            "type": str,  # "dialogue" or "direction"
                            "subtype": str,  # "dialogue"
                            "line": str,
                            "pre_directions": str,
                            "post_directions": str,
                            "extra": dict
                        }
                    ]
                }
            ]
        }
    ]
}

# GPT 프롬프트 템플릿
SYSTEM_PROMPT ="""
    
    당신은 연극 대본을 정확하게 JSON 형식으로 변환하는 전문가입니다.
    주어진 대본을 다음 규칙에 따라 JSON으로 변환해주세요:

    1. 모든 내용을 정확하게 포함하고, 어떤 내용도 생략하지 마세요.
    2. 대사는 캐릭터명과 함께 정확히 기록하세요.
    3. 지문은 type을 "direction"으로 처리하세요.
    4. 대사 전후의 괄호 안 지문은 pre_directions와 post_directions로 구분하세요.
    5. 모든 장면 설정과 무대 지시문을 정확히 포함하세요.
    6. 인물 뒤에 ":"이 꼭 있는 것은 아니니 고려해서 인물들을 처리해주세요.

    출력은 반드시 유효한 JSON 형식이어야 합니다.
"""

In [5]:
# [Cell 5] - GPT API 호출 함수 (디버깅 추가)
def parse_chunk_with_gpt(chunk: str) -> Dict:
    """GPT를 사용하여 대본 청크를 파싱합니다."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "developer",
                    "content": SYSTEM_PROMPT
                },
                {
                    "role": "user",
                    "content": f"다음 대본을 JSON으로 변환해주세요:\n\n{chunk}"
                }
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        
        result = json.loads(response.choices[0].message.content)
        print(f"\nParsed chunk result structure:")
        print(json.dumps(result, indent=2)[:200] + "...")  # 첫 200자만 출력
        return result
    
    except Exception as e:
        print(f"Error parsing chunk: {e}")
        print(f"Response content: {response.choices[0].message.content if 'response' in locals() else 'No response'}")
        return None

In [6]:
# [Cell 6] - 청크 결과 병합 함수 (디버깅 추가)
def merge_parsed_results(results: List[Dict]) -> Dict:
    """파싱된 청크들을 하나의 JSON으로 병합합니다."""
    if not results:
        return None
    
    print("Debugging merge process:")
    for i, result in enumerate(results):
        print(f"\nChunk {i+1} structure:")
        print(json.dumps(result, indent=2)[:200] + "...")  # 첫 200자만 출력
    
    # 첫 번째 결과를 기본으로 사용
    final_result = results[0].copy()
    
    try:
        # 이후 청크들의 내용을 병합
        for result in results[1:]:
            # 등장인물 정보 병합
            if "characters" in final_result and "characters" in result:
                existing_characters = {char["name"]: char for char in final_result["characters"]}
                for char in result["characters"]:
                    if char["name"] not in existing_characters:
                        final_result["characters"].append(char)
            
            # Act 정보 병합
            if "acts" in final_result and "acts" in result:
                for act in result["acts"]:
                    # 기존 act가 있는지 확인
                    existing_act = next(
                        (a for a in final_result["acts"] if a["act_number"] == act["act_number"]), 
                        None
                    )
                    
                    if existing_act:
                        # 기존 act에 scene들을 추가
                        existing_act["scenes"].extend(act["scenes"])
                    else:
                        # 새로운 act 추가
                        final_result["acts"].append(act)
    
        return final_result
    
    except Exception as e:
        print(f"\nError in merge process: {str(e)}")
        print(f"Error location: {e.__traceback__.tb_lineno}")
        raise

In [7]:
# [Cell 7] - 메인 실행 함수
def parse_script(script_path: str) -> Dict:
    """대본 파일을 읽고 JSON으로 변환합니다."""
    try:
        # 파일 읽기
        with open(script_path, 'r', encoding='utf-8') as f:
            script_text = f.read()
        
        # 토큰 수 확인
        total_tokens = count_tokens(script_text)
        print(f"Total tokens in script: {total_tokens}")
        
        if total_tokens <= 10000:
            # 단일 청크로 처리
            result = parse_chunk_with_gpt(script_text)
            return result
        else:
            # 청크로 분할하여 처리
            chunks = split_into_chunks(script_text)
            print(f"Split into {len(chunks)} chunks")
            
            results = []
            for i, chunk in enumerate(chunks):
                print(f"Processing chunk {i+1}/{len(chunks)}")
                result = parse_chunk_with_gpt(chunk)
                if result:
                    results.append(result)
            
            # 결과 병합
            return merge_parsed_results(results)
                
    except Exception as e:
        print(f"Error processing script: {e}")
        return None

In [8]:
# [Cell 8] - 결과 저장 함수
def save_parsed_script(parsed_script: Dict, output_path: str):
    """파싱된 스크립트를 JSON 파일로 저장합니다."""
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(parsed_script, f, ensure_ascii=False, indent=2)

In [9]:
script_path = "play_script/나무는 서서 죽는다.txt"
output_path = "play_script/나무는 서서 죽는다.json"   
    
result = parse_script(script_path)
if result:
    save_parsed_script(result, output_path)
    print(f"Successfully parsed and saved to {output_path}")
else:
    print("Failed to parse script")

Total tokens in script: 49049
Split into 5 chunks
Processing chunk 1/5

Parsed chunk result structure:
{
  "title": "\ub098\ubb34\ub294 \uc11c\uc11c \uc8fd\ub294\ub2e4 (Los \u00e1rboles mueren de pie)",
  "author": "Alejandro Casona",
  "characters": [
    "\uc18c\uc7a5",
    "\ubbf8\uac94",
    "\ub9c...
Processing chunk 2/5

Parsed chunk result structure:
{
  "play": [
    {
      "character": "\ub9c8\ub974\ub530",
      "lines": [
        {
          "text": "\ub2f9\uc2e0\uc774 \uc77c\uc790\ub9ac\ub97c \uc783\uc5c8\ub2e4\ub294 \uc0ac\uc2e4\uc744 \uc54...
Processing chunk 3/5

Parsed chunk result structure:
{
  "play": [
    {
      "character": "\ud560\uba38\ub2c8",
      "line": "\uc774\uc81c \ub208\uc774 \uc798 \uc548 \ubcf4\uc774\uac70\ub4e0, \ud558\uc9c0\ub9cc \uae30\uc5b5\uc774 \ub098\ub294\uad6c\u...
Processing chunk 4/5

Parsed chunk result structure:
{
  "play": {
    "title": "Untitled",
    "acts": [
      {
        "act_number": 3,
        "scenes": [
          {
        